<a href="https://colab.research.google.com/github/asmaakhaledd/PID-NN/blob/PID-Bolus/pid_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy pandas tensorflow

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import math

Load model

In [4]:
# Load the trained model
def load_model(model_path):
    pid_model = tf.keras.models.load_model(model_path, compile=False)
    pid_model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
    return pid_model

Preprocess time features (cyclic encoding)

In [5]:
# Preprocess time features (Use timestep as timestamp)
def preprocess_time_features(timestep, glucose, weight):
    # Using cyclic encoding for time, but we will use timestep as time (1, 2, 3,...)
    hour = timestep % 24
    minute = 0  # Simulate with minute 0 for simplicity
    time_sin = np.sin(2 * np.pi * hour / 24)
    time_cos = np.cos(2 * np.pi * hour / 24)
    return [glucose - 110, glucose - random.uniform(70, 180), weight, time_sin, time_cos]

Prepare PID data for inference

In [6]:
# Prepare PID data for inference
def prepare_pid_data(timestep, glucose, weight):
    # Preprocess real-time data for the PID model
    return np.array([preprocess_time_features(timestep, glucose, weight)])

Adjust basal insulin dosage based on glucose and weight

In [7]:
def adjust_basal_insulin(glucose_level, weight, Kp, Ki, Kd, previous_glucose, cumulative_error):
    # Basal insulin rate calculation based on weight
    basal_rate_per_kg = 0.5
    TDI = 0.55 * weight  # Total daily insulin
    basal_insulin_dosage = basal_rate_per_kg * TDI  # Basal insulin dosage (50% of TDI)
    hourly_basal_rate = basal_insulin_dosage / 24  # Hourly basal rate

    # Target glucose level
    target_glucose = 100

    # Calculate the error (difference between current glucose and target)
    error = glucose_level - target_glucose

    # Proportional (Kp) term: Respond to immediate error
    adjustment_factor_proportional = Kp * error

    # Integral (Ki) term: Cumulative error over time
    cumulative_error += error  # Update cumulative error
    if cumulative_error > 10:
        cumulative_error = 10
    elif cumulative_error < -10:
        cumulative_error = -10
    adjustment_factor_integral = Ki * cumulative_error

    # Derivative (Kd) term: Rate of change in glucose
    glucose_change_rate = glucose_level - previous_glucose  # Difference between current and previous glucose levels
    adjustment_factor_derivative = Kd * glucose_change_rate

    # Total adjusted basal rate including all PID terms
    adjusted_basal_rate = hourly_basal_rate + adjustment_factor_proportional + adjustment_factor_integral + adjustment_factor_derivative

    # Apply limits to the basal rate to prevent it from becoming too low or too high
    min_basal_rate = 0.3
    max_basal_rate = 1.5
    adjusted_basal_rate = max(min_basal_rate, min(adjusted_basal_rate, max_basal_rate))


    # Return the adjusted basal rate and the updated cumulative error for next use
    return adjusted_basal_rate, cumulative_error, glucose_level

Bolus Calculation

In [8]:
def calculate_bolus(weight, glucose_level, target_glucose, meal_carbs, insulin_type='rapid'):
    # Calculate Total Daily Insulin (TDI)
    TDI = weight * 0.55  # For weight in kg

    # Calculate Basal and Bolus
    basal_insulin = 0.4 * TDI
    bolus_insulin = 0.6 * TDI

    # Calculate the Correction Factor based on insulin type
    if insulin_type == 'rapid':
        correction_factor = 1500 / TDI


    # Calculate correction dose if BG is above target
    correction_dose = 0
    if glucose_level > target_glucose:
        correction_dose = (glucose_level - target_glucose) / correction_factor

    # Calculate carb-to-insulin ratio
    carb_to_insulin_ratio = 500 / TDI

    # Calculate bolus insulin based on carbs in the meal
    bolus_for_meal = meal_carbs / carb_to_insulin_ratio

    # Total bolus insulin for the meal including correction
    total_bolus = bolus_for_meal + correction_dose

    # Limit the bolus insulin to a safe range
    min_bolus = 0.5  # Minimum bolus insulin per meal
    max_bolus = 10.0  # Maximum bolus insulin per meal
    total_bolus = max(min_bolus, min(total_bolus, max_bolus))

    print(f"Bolus for meal: {bolus_for_meal:.2f} units, Correction dose: {correction_dose:.2f} units, Total bolus: {total_bolus:.2f} units")

    return total_bolus

Predict insulin dosage

In [9]:
def predict_insulin_dosage(pid_model, glucose, weight, timestep, previous_glucose, cumulative_error):
    # Prepare input data for PID model
    X_pid_test = prepare_pid_data(timestep, glucose, weight)

    # Make prediction using the trained PID model (predicting Kp, Ki, Kd)
    predicted_pid_gains = pid_model.predict(X_pid_test)

    # Extract predicted PID gains (Kp, Ki, Kd)
    Kp, Ki, Kd = predicted_pid_gains[0]

    # Adjust basal insulin based on the predicted PID gains and glucose level
    adjusted_basal_rate, cumulative_error, previous_glucose = adjust_basal_insulin(
        glucose, weight, Kp, Ki, Kd, previous_glucose, cumulative_error
    )

    # Print the result
    print(f"\nSample {timestep}:")
    print(f"Timestep: {timestep}")
    print(f"Glucose: {glucose:.2f} mg/dL")
    print(f"Adjusted Hourly Basal Insulin: {adjusted_basal_rate:.2f} U per hour")
    print(f"Weight: {weight} kg")
    print("-" * 50)

    # Return updated values to be used in the next timestep
    return previous_glucose, cumulative_error, glucose, adjusted_basal_rate

Dejavu-RM property specifications

In [10]:
def should_deliver_insulin(glucose):
    # Rule 1: Don't deliver insulin if the glucose sensor has failed
    if  glucose <= 0 or glucose > 1000:
        print(f"irregular reading detected (Glucose = {glucose}): Pausing insulin delivery.")
        return True

    # Rule 2: Do not deliver insulin if glucose < 50 mg/dL
    if glucose < 50:
        print(f"Hypoglycemia detected (Glucose = {glucose}): Pausing insulin delivery.")
        return True

     # Rule 3: Missing or unreadable glucose value
    if glucose is None or (isinstance(glucose, float) and math.isnan(glucose)):
        print("No glucose reading available: Pausing insulin delivery.")
        return True



    return False

parse_test_case

In [11]:
def parse_test_case(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    weight = None
    sim_time = None
    meals = []

    for line in lines:
        if "Body Weight" in line:
            weight = float(line.strip().split(':')[1].split()[0])
        elif "Simulation Time" in line:
            sim_time = int(line.strip().split(':')[1].split()[0])
        elif "Meal" in line and "Time" in line:
            parts = line.strip().split(',')
            time = int(parts[0].split(':')[1].strip().split()[0])
            carbs = float(parts[1].split(':')[1].strip().split()[0])
            meals.append((time, carbs))

    return weight, sim_time, meals

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Main

In [15]:
if __name__ == "__main__":
    # Path to your saved model
    model_path = '/content/drive/MyDrive/LSTM/opt_pid_tuning_model_2.h5'

    # Load the model
    pid_model = load_model(model_path)

    # Read the test case
    test_case_path = '/content/drive/MyDrive/LSTM/TestCases.txt'
    weight, sim_time, meals = parse_test_case(test_case_path)

    # Initialize variables
    previous_glucose = 100
    cumulative_error = 0

    # Set weight constant (can be updated as needed)
    weight = 70  # Constant weight

    # Initialize timestep
    timestep = 1

    # Variables for bolus calculation
    meal_carbs = 0

    meal_schedule = {time: carbs for time, carbs in meals}

    basal_rates = []
    bolus_insulin = []
    carb_intake = []


    while True:
      glucose = random.uniform(70, 180)
      if should_deliver_insulin(glucose):
        adjusted_basal_rate = 0


      elif timestep == 1 or timestep % 15 == 0:

        previous_glucose, cumulative_error, glucose, adjusted_basal_rate = predict_insulin_dosage(pid_model, glucose, weight, timestep, previous_glucose, cumulative_error)
        basal_rates.append(adjusted_basal_rate)


         # Check for bolus dose 10 minutes before meal
      if timestep + 10 in meal_schedule:
            meal_carbs = meal_schedule[timestep + 10]
            total_bolus = calculate_bolus(weight, glucose, target_glucose=110, meal_carbs=meal_carbs)
            bolus_insulin.append(total_bolus)
            carb_intake.append(meal_carbs)
      else:
            bolus_insulin.append(0)
            carb_intake.append(0)

        # Check for meal and calculate bolus (if a meal is administered)
        # if meal_carbs > 0:
        #       total_bolus = calculate_bolus(weight, glucose, target_glucose=110, meal_carbs=meal_carbs)
        #       bolus_insulin.append(total_bolus)
        #       carb_intake.append(meal_carbs)
        #       meal_carbs = 0
        # else:
        #       total_bolus = 0
        #       bolus_insulin.append(0)
        #       carb_intake.append(0)

        #   # Simulate meal intake (just as an example, change meal size over time)
        # if timestep == 30:  # Example of a meal at timestep 30
        #       meal_carbs = 100  # Example of 100g carbs meal
        # if timestep == 90:  # Example of a meal at timestep 50
        #       meal_carbs = 50  # Example of 50g carbs meal



      timestep+=1

      if timestep % 10 == 0:
        print(f"Processed Timestep {timestep}: Glucose: {glucose:.2f}, Basal Rate: {adjusted_basal_rate:.2f}")
      else:
        basal_rates.append(basal_rates[-1] if basal_rates else 1.0)  # Use last basal rate or default
      if timestep > 3500:
            break



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step

Sample 1:
Timestep: 1
Glucose: 159.95 mg/dL
Adjusted Hourly Basal Insulin: 1.50 U per hour
Weight: 70 kg
--------------------------------------------------
Processed Timestep 10: Glucose: 143.88, Basal Rate: 1.50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

Sample 15:
Timestep: 15
Glucose: 84.03 mg/dL
Adjusted Hourly Basal Insulin: 0.30 U per hour
Weight: 70 kg
--------------------------------------------------
Processed Timestep 20: Glucose: 167.78, Basal Rate: 0.30
Processed Timestep 30: Glucose: 121.69, Basal Rate: 0.30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

Sample 30:
Timestep: 30
Glucose: 90.77 mg/dL
Adjusted Hourly Basal Insulin: 0.30 U per hour
Weight: 70 kg
--------------------------------------------------
Processed Timestep 40: Glucose: 156.83, Basal Rate: 0.30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

Sample 45:
Timestep: 45
Glucose: 129.61 mg/dL
Adjusted Hourly Basal Insulin: 1.50 U per hour
Weight: 70 kg
--------------------------------------